# Reshaping, Reorganizing and Aggregation

In [1]:
# import pandas and NumPy
import pandas as pd
import numpy as np

# date and time functions
import datetime

# bring in matplotlib and draw inline
import matplotlib.pyplot as plt
%matplotlib inline

# Set some Pandas options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 8)
pd.set_option('precision', 3)

## Loading historical stock data from the web or from files

### From Web

In [2]:
# for the DataReader
import pandas.io.data as web

# start end end dates
start = datetime.datetime(2012, 1, 1)
end = datetime.datetime(2012, 12, 30)

# load the data
msft = web.DataReader("MSFT", 'yahoo', start, end)
aapl = web.DataReader("AAPL", 'yahoo', start, end)


/Users/kevin/python/anaconda/envs/timbr/lib/python2.7/site-packages/pandas/io/data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [3]:
# these save the data to file - optional for the examples
#msft.to_csv("msft.csv")
#aapl.to_csv("aapl.csv")

### From file

In [4]:
# read the Microsoft and Apple data from file
msft = pd.read_csv("msft.csv", index_col=0, parse_dates=True)
aapl = pd.read_csv("aapl.csv", index_col=0, parse_dates=True)

In [5]:
msft

             Open   High    Low  Close    Volume  Adj Close
Date                                                       
2012-01-03  26.55  26.96  26.39  26.77  64731500     23.462
2012-01-04  26.82  27.47  26.78  27.40  80516100     24.014
2012-01-05  27.38  27.73  27.29  27.68  56081400     24.259
2012-01-06  27.53  28.19  27.53  28.11  99455500     24.636
...           ...    ...    ...    ...       ...        ...
2012-12-24  27.20  27.25  27.00  27.06  20842400     24.387
2012-12-26  27.03  27.20  26.70  26.86  31631100     24.206
2012-12-27  26.89  27.09  26.57  26.96  39394000     24.297
2012-12-28  26.71  26.90  26.55  26.55  28239900     23.927

[249 rows x 6 columns]

### Organizing the data for the examples

In [6]:
msft[:3]

             Open   High    Low  Close    Volume  Adj Close
Date                                                       
2012-01-03  26.55  26.96  26.39  26.77  64731500     23.462
2012-01-04  26.82  27.47  26.78  27.40  80516100     24.014
2012-01-05  27.38  27.73  27.29  27.68  56081400     24.259

In [8]:
aapl[:3]

              Open    High     Low   Close    Volume  Adj Close
Date                                                           
2012-01-03  409.40  412.50  409.00  411.23  75555200     53.785
2012-01-04  410.00  414.68  409.28  413.44  65005500     54.074
2012-01-05  414.95  418.55  412.67  418.03  67817400     54.674

# Reorganizing and reshaping data

## Concatenating data in Multiple DataFrame objects

In [9]:
# get MSFT adjusted close data for Jan and Feb 2012
msftA01 = msft['2012-01'][['Adj Close']]
msftA02 = msft['2012-02'][['Adj Close']]
msftA01[:3]

            Adj Close
Date                 
2012-01-03     23.462
2012-01-04     24.014
2012-01-05     24.259

In [10]:
msftA02[:3]

            Adj Close
Date                 
2012-02-01     26.196
2012-02-02     26.249
2012-02-03     26.503

In [11]:
# combine the first three rows of each of msftA01 and msftA02
pd.concat([msftA01.head(3), msftA02.head(3)])

            Adj Close
Date                 
2012-01-03     23.462
2012-01-04     24.014
2012-01-05     24.259
2012-02-01     26.196
2012-02-02     26.249
2012-02-03     26.503

In [12]:
# Extract only the Jan 2012 AAPL values.  
aaplA01 = aapl['2012-01'][['Adj Close']]
# now concat the AAPL and MSFT Jan 2012 data
# there will be duplicate index labels
withDups = pd.concat([msftA01[:3], aaplA01[:3]])
withDups

            Adj Close
Date                 
2012-01-03     23.462
2012-01-04     24.014
2012-01-05     24.259
2012-01-03     53.785
2012-01-04     54.074
2012-01-05     54.674

In [10]:
# show the two records for data of 2012-01-03
withDups.ix['2012-01-03']

            Adj Close
Date                 
2012-01-03      24.42
2012-01-03      55.41

In [13]:
# demonstrate concat with a specification of the 
# stock tickets being part of the index
# this help disambiguate the duplicate dates using
# a hierarchal index
closes = pd.concat([msftA01[:3], aaplA01[:3]], 
                    keys=['MSFT', 'AAPL'])
closes

                 Adj Close
     Date                 
MSFT 2012-01-03     23.462
     2012-01-04     24.014
     2012-01-05     24.259
AAPL 2012-01-03     53.785
     2012-01-04     54.074
     2012-01-05     54.674

In [14]:
# extract just MSFT values using .ix
closes.ix['MSFT'][:3]

            Adj Close
Date                 
2012-01-03     23.462
2012-01-04     24.014
2012-01-05     24.259

In [15]:
# demonstrate concatenation using two DataFrame's
# that each have two columns.  pandas will align the
# data in columns by the column names (labels)
msftAV = msft[['Adj Close', 'Volume']]
aaplAV = aapl[['Adj Close', 'Volume']]
pd.concat([msftAV, aaplAV])

            Adj Close     Volume
Date                            
2012-01-03     23.462   64731500
2012-01-04     24.014   80516100
2012-01-05     24.259   56081400
2012-01-06     24.636   99455500
...               ...        ...
2012-12-24     68.637   43938300
2012-12-26     67.691   75609100
2012-12-27     67.963  113780100
2012-12-28     67.241   88569600

[498 rows x 2 columns]

In [16]:
# demonstrate concatenation with DataFrame objects
# that do not have the same set of columns
# this demonstrates pandas filling in NaN values
aaplA = aapl[['Adj Close']]
pd.concat([msftAV, aaplA])

            Adj Close     Volume
Date                            
2012-01-03     23.462  6.473e+07
2012-01-04     24.014  8.052e+07
2012-01-05     24.259  5.608e+07
2012-01-06     24.636  9.946e+07
...               ...        ...
2012-12-24     68.637        NaN
2012-12-26     67.691        NaN
2012-12-27     67.963        NaN
2012-12-28     67.241        NaN

[498 rows x 2 columns]

In [17]:
# perform an inner join on the DataFrame's
# since aaplA does not have a Volume column, pandas
# will not include that column in the result
pd.concat([msftAV, aaplA], join='inner')

            Adj Close
Date                 
2012-01-03     23.462
2012-01-04     24.014
2012-01-05     24.259
2012-01-06     24.636
...               ...
2012-12-24     68.637
2012-12-26     67.691
2012-12-27     67.963
2012-12-28     67.241

[498 rows x 1 columns]

In [18]:
# concat along the rows, causing duplicate columns to
# be created in the result
msftA = msft[['Adj Close']]
closes = pd.concat([msftA, aaplA], axis=1)
closes[:3]

            Adj Close  Adj Close
Date                            
2012-01-03     23.462     53.785
2012-01-04     24.014     54.074
2012-01-05     24.259     54.674

In [19]:
# concat along rows using two DataFrame objects with
# different number of rows. This demonstrates how
# NaN values will be filled in those rows for AAPL
# which only hase three rows as compared to 5 for MSFT
pd.concat([msftAV[:5], aaplAV[:3]], axis=1,
          keys=['MSFT', 'AAPL'])

                MSFT                AAPL           
           Adj Close    Volume Adj Close     Volume
Date                                               
2012-01-03    23.462  64731500    53.785  7.556e+07
2012-01-04    24.014  80516100    54.074  6.501e+07
2012-01-05    24.259  56081400    54.674  6.782e+07
2012-01-06    24.636  99455500       NaN        NaN
2012-01-09    24.312  59706800       NaN        NaN

In [20]:
# inner join can also be used along this axis
# this will not include rows with index labels that do
# not exist in both DataFrame objects
pd.concat([msftA[:5], aaplA[:3]], axis=1,
          join='inner', keys=['MSFT', 'AAPL'])

                MSFT      AAPL
           Adj Close Adj Close
Date                          
2012-01-03    23.462    53.785
2012-01-04    24.014    54.074
2012-01-05    24.259    54.674

In [21]:
# ignore indexes and just concatenate the data and
# have the result have a default integer index
pd.concat([msftA[:3], aaplA[:3]], ignore_index=True)

   Adj Close
0     23.462
1     24.014
2     24.259
3     53.785
4     54.074
5     54.674

## Merging DataFrame objects

In [22]:
# we will merge these two DataFrame objects, 
# so lets peek at the data to remind ourselves
# of what they contain
msftAR = msftA.reset_index()
msftVR = msft[['Volume']].reset_index()
msftAR[:3]

        Date  Adj Close
0 2012-01-03     23.462
1 2012-01-04     24.014
2 2012-01-05     24.259

In [23]:
msftVR[:3]

        Date    Volume
0 2012-01-03  64731500
1 2012-01-04  80516100
2 2012-01-05  56081400

In [24]:
# merge the two.  pandas finds the columns in common,
# in this case Date, and merges on that column and adds
# a column for all the other columns in both DataFrame's
msftCVR = pd.merge(msftAR, msftVR)
msftCVR[:5]

        Date  Adj Close    Volume
0 2012-01-03     23.462  64731500
1 2012-01-04     24.014  80516100
2 2012-01-05     24.259  56081400
3 2012-01-06     24.636  99455500
4 2012-01-09     24.312  59706800

In [27]:
# we will demonstrate join semantics using this DataFrame
msftAR0_5 = msftAR[0:5]
msftAR0_5

        Date  Adj Close
0 2012-01-03     23.462
1 2012-01-04     24.014
2 2012-01-05     24.259
3 2012-01-06     24.636
4 2012-01-09     24.312

In [28]:
# and also this one
msftVR2_4 = msftVR[2:4]
msftVR2_4

        Date    Volume
2 2012-01-05  56081400
3 2012-01-06  99455500

In [29]:
# merge semantics using default inner join
pd.merge(msftAR0_5, msftVR2_4)

        Date  Adj Close    Volume
0 2012-01-05     24.259  56081400
1 2012-01-06     24.636  99455500

In [30]:
# same joing but using
pd.merge(msftAR0_5, msftVR2_4, how='outer')

        Date  Adj Close     Volume
0 2012-01-03     23.462        NaN
1 2012-01-04     24.014        NaN
2 2012-01-05     24.259  5.608e+07
3 2012-01-06     24.636  9.946e+07
4 2012-01-09     24.312        NaN

## Pivoting

In [31]:
# need to insert Symbol column before combining
msft.insert(0, 'Symbol', 'MSFT')
aapl.insert(0, 'Symbol', 'AAPL')

# concatenate the MSFT and AAPL data
# index will consist of the Date column, which we will sort
combined = pd.concat([msft, aapl]).sort_index()

# this pushes the index into a column and resets to a 
# default integer index
s4p = combined.reset_index();
s4p[:5]

        Date Symbol    Open    High     Low   Close    Volume  Adj Close
0 2012-01-03   MSFT   26.55   26.96   26.39   26.77  64731500     23.462
1 2012-01-03   AAPL  409.40  412.50  409.00  411.23  75555200     53.785
2 2012-01-04   MSFT   26.82   27.47   26.78   27.40  80516100     24.014
3 2012-01-04   AAPL  410.00  414.68  409.28  413.44  65005500     54.074
4 2012-01-05   MSFT   27.38   27.73   27.29   27.68  56081400     24.259

In [32]:
# pivot Date into the Index, make the columns match the
# unique values in the Symbol column, and the values 
# will be the AdjClose values
closes = s4p.pivot(index='Date', columns='Symbol', 
                   values='Adj Close')
closes[:3]

Symbol        AAPL    MSFT
Date                      
2012-01-03  53.785  23.462
2012-01-04  54.074  24.014
2012-01-05  54.674  24.259

## Stacking and Unstacking

In [33]:
# stack the first level of columns into the index
# essentially, moves AAPL and MSFT into the index
# leaving a single colum which is the AdjClose values
stackedCloses = closes.stack()
stackedCloses

Date        Symbol
2012-01-03  AAPL      53.785
            MSFT      23.462
2012-01-04  AAPL      54.074
            MSFT      24.014
                       ...  
2012-12-27  AAPL      67.963
            MSFT      24.297
2012-12-28  AAPL      67.241
            MSFT      23.927
dtype: float64

In [34]:
# using .ix we can retrieve close values by
# specifying both the date and ticker
stackedCloses.ix['2012-01-03', 'AAPL']

53.784500000000001

In [35]:
# lookup on just the date, which will give us two values
# one each for AAPL and MSFT.  
stackedCloses.ix['2012-01-03']

Symbol
AAPL    53.785
MSFT    23.462
dtype: float64

In [36]:
# this looks up all values for the MSFT symbol
stackedCloses.ix[:, 'MSFT']

Date
2012-01-03    23.462
2012-01-04    24.014
2012-01-05    24.259
2012-01-06    24.636
               ...  
2012-12-24    24.387
2012-12-26    24.206
2012-12-27    24.297
2012-12-28    23.927
dtype: float64

In [37]:
# pivots the last level of the index back into a column
unstackedCloses = stackedCloses.unstack()
unstackedCloses[:3]

Symbol        AAPL    MSFT
Date                      
2012-01-03  53.785  23.462
2012-01-04  54.074  24.014
2012-01-05  54.674  24.259

## Melting

In [38]:
# melt making id_vars of Date and Symbol, making the 
# column names the variable and the for each the value
melted = pd.melt(s4p, id_vars=['Date', 'Symbol'])
melted[:5]

        Date Symbol variable   value
0 2012-01-03   MSFT     Open   26.55
1 2012-01-03   AAPL     Open  409.40
2 2012-01-04   MSFT     Open   26.82
3 2012-01-04   AAPL     Open  410.00
4 2012-01-05   MSFT     Open   27.38

In [39]:
# extract the values for the data for MSFT on 2012-01-03
melted[(melted.Date=='2012-01-03') & (melted.Symbol=='MSFT')]

           Date Symbol   variable      value
0    2012-01-03   MSFT       Open  2.655e+01
498  2012-01-03   MSFT       High  2.696e+01
996  2012-01-03   MSFT        Low  2.639e+01
1494 2012-01-03   MSFT      Close  2.677e+01
1992 2012-01-03   MSFT     Volume  6.473e+07
2490 2012-01-03   MSFT  Adj Close  2.346e+01

# Grouping and aggregation

## Splitting

In [40]:
# construct a DataFrame to demonstrate splitting
# extract from combined the Symbol and AdjClose, and reset the index
s4g = combined[['Symbol', 'Adj Close']].reset_index()
# now, add two columns, year and month, using the year and month
# portions of the data as integers
s4g.insert(1, 'Year', pd.DatetimeIndex(s4g['Date']).year)
s4g.insert(2, 'Month',pd.DatetimeIndex(s4g['Date']).month)
s4g[:5]

        Date  Year  Month Symbol  Adj Close
0 2012-01-03  2012      1   MSFT     23.462
1 2012-01-03  2012      1   AAPL     53.785
2 2012-01-04  2012      1   MSFT     24.014
3 2012-01-04  2012      1   AAPL     54.074
4 2012-01-05  2012      1   MSFT     24.259

In [41]:
# group by the Symbol column
s4g.groupby('Symbol')

In [42]:
# group again, but save the result this time
grouped = s4g.groupby('Symbol')
# the groupby object has a property groups, which shows how
# all rows will in mapped into the groups.   
# the type of this object is a python dict
type(grouped.groups)

dict

In [43]:
# show the mappings of rows to groups
grouped.groups

{'AAPL': [1,
  3,
  5,
  7,
  9,
  11,
  13,
  14,
  16,
  18,
  20,
  23,
  25,
  27,
  29,
  30,
  33,
  34,
  37,
  38,
  41,
  43,
  45,
  46,
  48,
  50,
  53,
  54,
  56,
  58,
  61,
  63,
  64,
  67,
  69,
  71,
  72,
  75,
  77,
  79,
  81,
  82,
  84,
  86,
  89,
  91,
  92,
  94,
  97,
  99,
  100,
  102,
  104,
  107,
  109,
  111,
  112,
  114,
  116,
  118,
  120,
  123,
  125,
  126,
  128,
  130,
  132,
  134,
  137,
  138,
  140,
  143,
  145,
  146,
  148,
  150,
  153,
  155,
  156,
  159,
  161,
  162,
  165,
  166,
  169,
  171,
  173,
  175,
  176,
  179,
  181,
  183,
  184,
  186,
  189,
  190,
  193,
  195,
  197,
  199,
  200,
  203,
  205,
  207,
  209,
  211,
  212,
  214,
  216,
  219,
  221,
  223,
  224,
  227,
  228,
  230,
  232,
  234,
  237,
  238,
  240,
  242,
  244,
  247,
  248,
  250,
  252,
  254,
  257,
  259,
  261,
  263,
  265,
  267,
  269,
  271,
  273,
  275,
  277,
  278,
  280,
  282,
  285,
  287,
  289,
  290,
  293,
  295,
  296,
  29

In [44]:
# these report the number of groups that resulted from
# the grouping
len(grouped), grouped.ngroups

(2, 2)

In [45]:
# this function will print the contents of a group
def print_groups (groupobject):
    for name, group in groupobject:
        print name
        print group.head()

In [46]:
# examine our resulting groups
print_groups(grouped)

AAPL
        Date  Year  Month Symbol  Adj Close
1 2012-01-03  2012      1   AAPL     53.785
3 2012-01-04  2012      1   AAPL     54.074
5 2012-01-05  2012      1   AAPL     54.674
7 2012-01-06  2012      1   AAPL     55.245
9 2012-01-09  2012      1   AAPL     55.158
MSFT
        Date  Year  Month Symbol  Adj Close
0 2012-01-03  2012      1   MSFT     23.462
2 2012-01-04  2012      1   MSFT     24.014
4 2012-01-05  2012      1   MSFT     24.259
6 2012-01-06  2012      1   MSFT     24.636
8 2012-01-09  2012      1   MSFT     24.312


In [47]:
# .size will tell us the count of items in each group
grouped.size()

Symbol
AAPL    249
MSFT    249
dtype: int64

In [48]:
# a specific group can be retrieved using .get_group()
# which returns a DataFrame representing the specified group
grouped.get_group('MSFT')

          Date  Year  Month Symbol  Adj Close
0   2012-01-03  2012      1   MSFT     23.462
2   2012-01-04  2012      1   MSFT     24.014
4   2012-01-05  2012      1   MSFT     24.259
6   2012-01-06  2012      1   MSFT     24.636
..         ...   ...    ...    ...        ...
491 2012-12-24  2012     12   MSFT     24.387
493 2012-12-26  2012     12   MSFT     24.206
495 2012-12-27  2012     12   MSFT     24.297
496 2012-12-28  2012     12   MSFT     23.927

[249 rows x 5 columns]

In [50]:
# group by three different fields and print the result
mcg = s4g.groupby(['Symbol', 'Year', 'Month'])
print_groups(mcg)

('AAPL', 2012, 1)
        Date  Year  Month Symbol  Adj Close
1 2012-01-03  2012      1   AAPL     53.785
3 2012-01-04  2012      1   AAPL     54.074
5 2012-01-05  2012      1   AAPL     54.674
7 2012-01-06  2012      1   AAPL     55.245
9 2012-01-09  2012      1   AAPL     55.158
('AAPL', 2012, 2)
         Date  Year  Month Symbol  Adj Close
41 2012-02-01  2012      2   AAPL     59.665
43 2012-02-02  2012      2   AAPL     59.525
45 2012-02-03  2012      2   AAPL     60.121
46 2012-02-06  2012      2   AAPL     60.682
48 2012-02-07  2012      2   AAPL     61.318
('AAPL', 2012, 3)
         Date  Year  Month Symbol  Adj Close
81 2012-03-01  2012      3   AAPL     71.211
82 2012-03-02  2012      3   AAPL     71.304
84 2012-03-05  2012      3   AAPL     69.732
86 2012-03-06  2012      3   AAPL     69.352
89 2012-03-07  2012      3   AAPL     69.409
('AAPL', 2012, 4)
          Date  Year  Month Symbol  Adj Close
125 2012-04-02  2012      4   AAPL     80.910
126 2012-04-03  2012      4   AA

In [51]:
# set the index of the data to be the following three fields
# we are creating a multiindex
mi = s4g.set_index(['Symbol', 'Year', 'Month'])
mi

                        Date  Adj Close
Symbol Year Month                      
MSFT   2012 1     2012-01-03     23.462
AAPL   2012 1     2012-01-03     53.785
MSFT   2012 1     2012-01-04     24.014
AAPL   2012 1     2012-01-04     54.074
...                      ...        ...
            12    2012-12-27     67.963
MSFT   2012 12    2012-12-27     24.297
            12    2012-12-28     23.927
AAPL   2012 12    2012-12-28     67.241

[498 rows x 2 columns]

In [52]:
# now we can group based upon values in the actual index
# the following groups by level 0 of the index (Month)
mig_l1 = mi.groupby(level=0)
print_groups(mig_l1)

AAPL
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 1     2012-01-03     53.785
            1     2012-01-04     54.074
            1     2012-01-05     54.674
            1     2012-01-06     55.245
            1     2012-01-09     55.158
MSFT
                        Date  Adj Close
Symbol Year Month                      
MSFT   2012 1     2012-01-03     23.462
            1     2012-01-04     24.014
            1     2012-01-05     24.259
            1     2012-01-06     24.636
            1     2012-01-09     24.312


In [53]:
# group by three levels in the index using their names
mig_l12 = mi.groupby(level=['Symbol', 'Year', 'Month'])
print_groups(mig_l12)

('AAPL', 2012, 1)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 1     2012-01-03     53.785
            1     2012-01-04     54.074
            1     2012-01-05     54.674
            1     2012-01-06     55.245
            1     2012-01-09     55.158
('AAPL', 2012, 2)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 2     2012-02-01     59.665
            2     2012-02-02     59.525
            2     2012-02-03     60.121
            2     2012-02-06     60.682
            2     2012-02-07     61.318
('AAPL', 2012, 3)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 3     2012-03-01     71.211
            3     2012-03-02     71.304
            3     2012-03-05     69.732
            3     2012-03-06     69.352
            3     2012-03-07     69.409
('AAPL', 2012, 4)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2

# Aggregation

In [54]:
# this will apply the mean function to each group
mig_l12.agg(np.mean)

                   Adj Close
Symbol Year Month           
AAPL   2012 1         56.053
            2         65.077
            3         75.532
            4         79.259
...                      ...
MSFT   2012 9         27.515
            10        25.981
            11        24.983
            12        24.314

[24 rows x 1 columns]

In [55]:
# example of groupby that also ignores the index
# resulting in a default integer index
# this also has the mean function applied
s4g.groupby(['Symbol', 'Year', 'Month'], 
            as_index=False).agg(np.mean)[:5]

  Symbol  Year  Month  Adj Close
0   AAPL  2012      1     56.053
1   AAPL  2012      2     65.077
2   AAPL  2012      3     75.532
3   AAPL  2012      4     79.259
4   AAPL  2012      5     73.853

In [56]:
# apply multiple functions to each group in one call
mig_l12.agg([np.mean, np.std])

                  Adj Close       
                       mean    std
Symbol Year Month                 
AAPL   2012 1        56.053  1.745
            2        65.077  3.465
            3        75.532  4.040
            4        79.259  2.973
...                     ...    ...
MSFT   2012 9        27.515  0.414
            10       25.981  0.640
            11       24.983  0.962
            12       24.314  0.350

[24 rows x 2 columns]